## **Ipthon Script to calculate NACorrection and Fractional Enrichment for a demo dataset.**

Prerequisite Knowledge:

**Natural Abundance Correction**- Natural abundance (NA) refers to the abundance of isotopes of a chemical element as naturally found on the planet. While performing analysis,the observed intensity contains contribution from isotopic natural abundance that needs to be corrected. This process is referred as NA Correction.

**Pool Total**- Sum of the intensities of all different number of labeled atoms of the isotope element is called pool total.

**Fractional enrichment**- Normalization of intensities of a metabolite between the range of 0 to 1.

**Welcome to the interactive Polly IPython Notebook.**

With this interactive Polly notebook you would be able to calculate NA Corrected intensities as well as fractional enrichment for LCMS/MS input file. Information on some functions used:

 - corna- package which looks into NA Correction.
 - msms.csv - demo raw_intensity file.
 - multiquant_parser.merge_mq_metadata - merge multiquant files and metadata files
 - multiquant_parser.add_mass_and_no_of_atoms_info_frm_label - from label column add information of molecular mass,    isotopic mass, total number of atoms, number of labeled atoms for parent as well as isotope fragment.
 - fractional_enrichment - Calculates fractional enrichment for the NA Corrected dataframe.

In [83]:
import pandas as pd
import numpy as np
import re

import corna.constants as const
from corna.helpers import get_isotope_na, replace_negatives_in_column
from corna.inputs.column_conventions import multiquant 
from corna.inputs import multiquant_parser
from corna.postprocess import fractional_enrichment
from corna.algorithms.background_correction import background_correction

**Defining the input files path and Natural Abundance values of elements.**

In [84]:
raw_df= pd.read_csv('Raw_intensity_file.csv')
metadata_df= pd.read_csv('metadata_mq_file.csv')
sample_metadata = pd.read_csv('sample_metadata.csv')
isBackground= False
isotope_dict= const.ISOTOPE_NA_MASS
REQUIRED_COL= [multiquant.FORMULA, multiquant.LABEL, multiquant.NAME, multiquant.SAMPLE, multiquant.COHORT,
                     multiquant.MQ_FRAGMENT, multiquant.INTENSITY, multiquant.PARENT_FORM,const.NA_CORRECTED_COL, 
                        const.BACKGROUND_WITH_ZERO, const.BACKGROUND_CORRECTED]

**Merge the raw_intensity dataframe with the metadata and sample metadata(if background correction to be performed).**

In [85]:
msms_df, list_of_replicates = multiquant_parser.merge_mq_metadata(raw_df, metadata_df, sample_metadata)

In [86]:
final_df = background_correction(msms_df, list_of_replicates, isotope_dict=const.ISOTOPE_NA_MASS)
final_df = replace_negatives_in_column(final_df,const.BACKGROUND_WITH_ZERO, const.BACKGROUND_CORRECTED)
print final_df

     Component Name                                             Sample  \
0    Taurine 124/80  Brandon_hepatocytes_31May18-CDK1,2,5 lac10mM [...   
8    Taurine 124/80  Brandon_hepatocytes_31May18-CDK4,6 lac 10mM [U...   
15   Taurine 124/80  Brandon_hepatocytes_31May18-CDK1,2,5 lac10mM [...   
23   Taurine 124/80  Brandon_hepatocytes_31May18-CDK1,2,5 lac10mM [...   
29   Taurine 124/80  Brandon_hepatocytes_31May18-DMSO lac10mM [U13C...   
36   Taurine 124/80  Brandon_hepatocytes_31May18-CDK4,6 lac 10mM [U...   
43   Taurine 124/80  Brandon_hepatocytes_31May18-DMSO lac10mM [U13C...   
49   Taurine 124/80  Brandon_hepatocytes_31May18-CDK1,2,5 lac10mM [...   
55   Taurine 124/80  Brandon_hepatocytes_31May18-CDK1,2,5 lac10mM [...   
61   Taurine 124/80  Brandon_hepatocytes_31May18-DMSO lac10mM NA (1...   
69   Taurine 124/80  Brandon_hepatocytes_31May18-Insulin lac 10mM [...   
76   Taurine 124/80  Brandon_hepatocytes_31May18-DMSO lac10mM [U13C...   
83   Taurine 124/80  Brandon_hepatocyt

**From Label column add information of molecular mass, isotopic mass, total number of atoms, number of labeled atoms for parent as well as isotope fragment.**

In [87]:
isotracer = msms_df[multiquant.ISOTRACER].unique()
intensity_col= const.BACKGROUND_WITH_ZERO

**Get Natural abundance value of the isotracer present in the compound(na).**

In [88]:
final_df[const.NA_CORRECTED_COL]=0.0
output_df= pd.DataFrame()
metab_dict={}

na= get_isotope_na(isotracer[0], isotope_dict)

**PARENT_NUM_ ATOMS - Total number of atoms of the isotracer element in parent formula.**

**DAUGHTER_NUM_ATOMS - Total number of atoms of the isotracer element in daughter formula.**

**PARENT_NUM_LABELED_ATOMS - number of labeled atoms ofisotracer element in parent formula.**

**DAUGHTER_NUM_LABELED_ATOMS - number of labeled atoms ofisotracer element in daughter formula.**

In [89]:
final_df['A']=(1 + na * (final_df[const.PARENT_NUM_ATOMS]-final_df[const.PARENT_NUM_LABELED_ATOMS]))
final_df['B']= na * ((final_df[const.PARENT_NUM_ATOMS]-final_df[const.DAUGHTER_NUM_ATOMS]) -\
                         (final_df[const.PARENT_NUM_LABELED_ATOMS]-final_df[const.DAUGHTER_NUM_LABELED_ATOMS]-1))
final_df['C']=  na * (final_df[const.DAUGHTER_NUM_ATOMS]-final_df[const.DAUGHTER_NUM_LABELED_ATOMS]+1)

**Drop columns not required for processing.**

In [90]:
final_df.drop([const.PARENT_MASS_MOL, const.DAUGHTER_MASS_MOL, const.PARENT_NUM_ATOMS,
                const.DAUGHTER_NUM_ATOMS, const.DAUGHTER_NUM_LABELED_ATOMS, const.PARENT_NUM_LABELED_ATOMS], axis=1, inplace=True)

**Create metabolite : intensity dictionary of the form:**

        {'SAMPLE 2_10':{
            (191, 111): 2345.75, (192, 111):5644.847
            }
        }

In [91]:
for samp in final_df.Sample.unique():
    
    metab_df = final_df[final_df[multiquant.SAMPLE]==samp]
    frag_dict={}
    for index, row in metab_df.iterrows():
        frag_dict[(row[const.PARENT_MASS_ISO],row[const.DAUGHTER_MASS_ISO])]=row[intensity_col]
    
    metab_dict[samp]= frag_dict

**In each sample correct the intensities of daughter fragment one by one using the intensity of M+0 isotopolgue.**

In [92]:
for samp in final_df.Sample.unique():
    metab_df = final_df[final_df[multiquant.SAMPLE]==samp]
    frag= metab_dict[samp]  
     
    for index, row in metab_df.iterrows():
        m_n= row[const.DAUGHTER_MASS_ISO]
        m_1_n= row[const.PARENT_MASS_ISO]-1
        m_n_1= row[const.DAUGHTER_MASS_ISO]-1
        intensity_m_n= row[intensity_col]
        try:
            intensity_m_1_n= frag[m_1_n, m_n]
        except KeyError:
            intensity_m_1_n=0
        try:
            intensity_m_1_n_1= frag[m_1_n, m_n_1]
        except KeyError:
            intensity_m_1_n_1= 0
        
        corrected= intensity_m_n * row['A']  - intensity_m_1_n * row['B'] -\
                                                intensity_m_1_n_1 * row['C']
        metab_df.set_value(index=index, col=const.NA_CORRECTED_COL, value= corrected)

    output_df=output_df.append(metab_df) 

**Filter the output dataframe to extract the required columns.**

In [93]:
output_df= output_df.filter(REQUIRED_COL)
output_df = replace_negatives_in_column(output_df,const.NA_CORRECTED_WITH_ZERO, const.NA_CORRECTED_COL)
print output_df

    Formula           Label            Name  \
0       O3S  C13_124.0_80.0  Taurine 124/80   
1       PO3  C13_167.0_79.0      PEP 167/79   
2       PO3  C13_168.0_79.0      PEP 167/79   
3       PO3  C13_169.0_79.0      PEP 167/79   
4       PO3  C13_170.0_79.0      PEP 167/79   
5     PO4H2  C13_169.0_97.0     DHAP 169/97   
6     PO4H2  C13_170.0_97.0     DHAP 169/97   
7     PO4H2  C13_171.0_97.0     DHAP 169/97   
8       O3S  C13_124.0_80.0  Taurine 124/80   
9       PO3  C13_167.0_79.0      PEP 167/79   
10      PO3  C13_168.0_79.0      PEP 167/79   
11      PO3  C13_170.0_79.0      PEP 167/79   
12    PO4H2  C13_169.0_97.0     DHAP 169/97   
13    PO4H2  C13_170.0_97.0     DHAP 169/97   
14    PO4H2  C13_171.0_97.0     DHAP 169/97   
15      O3S  C13_124.0_80.0  Taurine 124/80   
16      PO3  C13_167.0_79.0      PEP 167/79   
17      PO3  C13_168.0_79.0      PEP 167/79   
18      PO3  C13_169.0_79.0      PEP 167/79   
19      PO3  C13_170.0_79.0      PEP 167/79   
20    PO4H2  

**Calculate Fractional Enrichment**

In [94]:
fractional_enriched_df = fractional_enrichment(output_df)
print fractional_enriched_df

                                                Sample            Name  \
0    Brandon_hepatocytes_31May18-CDK1,2,5 lac10mM [...  Taurine 124/80   
1    Brandon_hepatocytes_31May18-CDK1,2,5 lac10mM [...      PEP 167/79   
2    Brandon_hepatocytes_31May18-CDK1,2,5 lac10mM [...      PEP 167/79   
3    Brandon_hepatocytes_31May18-CDK1,2,5 lac10mM [...      PEP 167/79   
4    Brandon_hepatocytes_31May18-CDK1,2,5 lac10mM [...      PEP 167/79   
5    Brandon_hepatocytes_31May18-CDK1,2,5 lac10mM [...     DHAP 169/97   
6    Brandon_hepatocytes_31May18-CDK1,2,5 lac10mM [...     DHAP 169/97   
7    Brandon_hepatocytes_31May18-CDK1,2,5 lac10mM [...     DHAP 169/97   
8    Brandon_hepatocytes_31May18-CDK4,6 lac 10mM [U...  Taurine 124/80   
9    Brandon_hepatocytes_31May18-CDK4,6 lac 10mM [U...      PEP 167/79   
10   Brandon_hepatocytes_31May18-CDK4,6 lac 10mM [U...      PEP 167/79   
11   Brandon_hepatocytes_31May18-CDK4,6 lac 10mM [U...      PEP 167/79   
12   Brandon_hepatocytes_31May18-CDK4,

In [95]:
df= pd.merge(output_df, fractional_enriched_df, on=['Label', 'Sample', 'Name', 'Formula'])
print df

    Formula           Label            Name  \
0       O3S  C13_124.0_80.0  Taurine 124/80   
1       PO3  C13_167.0_79.0      PEP 167/79   
2       PO3  C13_168.0_79.0      PEP 167/79   
3       PO3  C13_169.0_79.0      PEP 167/79   
4       PO3  C13_170.0_79.0      PEP 167/79   
5     PO4H2  C13_169.0_97.0     DHAP 169/97   
6     PO4H2  C13_170.0_97.0     DHAP 169/97   
7     PO4H2  C13_171.0_97.0     DHAP 169/97   
8       O3S  C13_124.0_80.0  Taurine 124/80   
9       PO3  C13_167.0_79.0      PEP 167/79   
10      PO3  C13_168.0_79.0      PEP 167/79   
11      PO3  C13_170.0_79.0      PEP 167/79   
12    PO4H2  C13_169.0_97.0     DHAP 169/97   
13    PO4H2  C13_170.0_97.0     DHAP 169/97   
14    PO4H2  C13_171.0_97.0     DHAP 169/97   
15      O3S  C13_124.0_80.0  Taurine 124/80   
16      PO3  C13_167.0_79.0      PEP 167/79   
17      PO3  C13_168.0_79.0      PEP 167/79   
18      PO3  C13_169.0_79.0      PEP 167/79   
19      PO3  C13_170.0_79.0      PEP 167/79   
20    PO4H2  